**Principal Component Analysis**

You will implement dimensionality reduction with PCA.  

1). Read iris_dataset.csv (4 features, hence 4 PCs)

2). Find the principal components

3). Recontruct the dataset (X_hat)

4). Determine the accuracy of X_hat for 1 PC and 4 PCs using LDA classifier (provided below)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as LA


# Load data - 150 observations, 4 features, 3 classes, 
df = pd.read_csv("iris_dataset.csv", header=None)
print(df.describe())
data = df.values

                0           1           2           3           4
count  150.000000  150.000000  150.000000  150.000000  150.000000
mean     5.843333    3.057333    3.758000    1.199333    2.000000
std      0.828066    0.435866    1.765298    0.762238    0.819232
min      4.300000    2.000000    1.000000    0.100000    1.000000
25%      5.100000    2.800000    1.600000    0.300000    1.000000
50%      5.800000    3.000000    4.350000    1.300000    2.000000
75%      6.400000    3.300000    5.100000    1.800000    3.000000
max      7.900000    4.400000    6.900000    2.500000    3.000000


In [2]:
## Setup

# Shuffle data randomly
shuffled_data = data;
np.random.shuffle(shuffled_data)
X = shuffled_data[:,0:4]  # 150x4


In [3]:
def evaluate_performance(Xhat, Num_PC, recon_error):
  '''
    Inputs:
      Xhat : reconstructed data set, size 150x4
      Num_PC : number of PCs to be used for reconstruction
      recon_error : reconstruction error
  '''
  
  ##################################
  # Evaluate performance using LDA #
  ##################################
  
  from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
  from sklearn.model_selection import cross_val_score

  no_dim = Num_PC  # number of dimensions
  #X_train = X[:,0:no_dim]           # original dataset
  X_train = Xhat[:,0:Num_PC]        # dimensionally reduced dataset
  y_train = data[:,4]

  model_mean_scores = []
  model = LinearDiscriminantAnalysis().fit(X_train, y_train)
  scores = cross_val_score(model, X_train, y_train, cv=10)
  model_mean_scores.append(np.mean(scores))

  # Reconstruction error
  print('Reconstruction error = {0:0.6f} with {1:1d} PCs, average accuracy = {2:0.4f}'
     .format(recon_error, Num_PC, model_mean_scores[0]))

In [4]:
def create_pc(X, Num_PC):
  '''
    Inputs: 
      X : original data set, size 150x4
      Num_PC : number of PC's to be used to recover Xhat
    Outputs:
      PC : principal components, size 150x4
      Xhat : reconstructed data set, size 150x4
      recon_error : reconstruction error
  '''

  ##################################
  ## Create Principal Components
  ##################################

  ## Zero out mean of data
  # Hint: use np.mean with axis=0
  # Your code goes here ...
  X_mean = np.copy(X)
  X_mean[:]=np.mean(X_mean,axis=0)
  X_mean[:,0] = 0

  ## Find the covariance matrix (4x4)
  # Hint: use np.cov and np.transpose
  # Your code goes here ...
  X_cov = np.subtract(X,X_mean)
  X_cov = np.cov(np.transpose(X_cov))

  ## Apply eigendecomposition to the covariance matrix
  ## Eigenvalues w (4x1); eigenvector V (4x4)
  # Hint: w,V = LA.eig(...) 
  #   where V is the eigenmatrix (4x4) and w is the eigenvalues(4x1)
  # Your code goes here ...
  w,V = LA.eig(X_cov)


  ## Compute principal components
  ## PC (150x4)
  # Hint: use np.matmul
  # Your code goes here ...
  PC = np.matmul(np.subtract(X,X_mean),V)


  ########################################################
  # Reconstruct X_hat & compute the reconstruction error
  ########################################################
  ## Compute Xhat (150x4)
  ## Num_PC : number of PC's used to reconstruct X_hat
  # Hint: use np.matmul, np.transpose
  # Your code goes here ...
  if(Num_PC == 1):
    X_hat = np.add(X_mean,PC)
  else:
    X_hat = np.add(X_mean,np.matmul(PC,np.transpose(V)))


  # Reconstruction error
  #    (l2norm(X) - l2norm(Xhat)) / l2norm(X)
  # Hint: use np.linalg.norm
  # Your code goes here ...

  recon_err = (np.linalg.norm(X) - np.linalg.norm(X_hat)) / np.linalg.norm(X)

  return PC, X_hat, recon_err

In [5]:
PC, X_hat, recon_err = create_pc(X,1)
evaluate_performance(X_hat, 1, recon_err)

PC, X_hat, recon_err = create_pc(X,4)
evaluate_performance(X_hat, 4, recon_err)



Reconstruction error = 0.453473 with 1 PCs, average accuracy = 0.9267
Reconstruction error = -0.000000 with 4 PCs, average accuracy = 0.9800
